In [ ]:
#r "nuget:Zeiss.Micro.LibCzi.Net,0.1.0-alpha"
#r "nuget: SkiaSharp"
#r "nuget: SixLabors.ImageSharp"
using SkiaSharp;
using SixLabors.ImageSharp;
using Zeiss.Micro.LibCzi.Net.Interface;
using System.Text.Json;

### Pixel Conversion (LibCZI IBitmap -> SKBitmap)

In [ ]:
SKBitmap Gray16ToBgra8888(int Width, int Height, int Stride, IntPtr bitmapData, bool normalize = true) {
    var dstInfo = new SKImageInfo(Width, Height, SKColorType.Bgra8888, SKAlphaType.Opaque);
    var dstBitmap = new SKBitmap(dstInfo);

    unsafe
    {
        byte* dstBase = (byte*)dstBitmap.GetPixels().ToPointer();
        byte* srcBase = (byte*)bitmapData;

        int dstStride = dstBitmap.RowBytes;

        ushort minValue = ushort.MaxValue;
        ushort maxValue = ushort.MinValue;

        if (normalize)
        {
            for (int y = 0; y < Height; y++) {
                ushort* srcRow = (ushort*)(srcBase + y * Stride);
                for (int x = 0; x < Width; x++) {
                    ushort value = srcRow[x];
                    if (value < minValue) minValue = value;
                    if (value > maxValue) maxValue = value;
                }
            }
        }


        for (int y = 0; y < Height; y++) {
            ushort* srcRow = (ushort*)(srcBase + y * Stride);
            byte* dstRow = dstBase + y * dstStride;

            for (int x = 0; x < Width; x++) {
                ushort value = srcRow[x];
                byte gray = 0;
                if (normalize)
                {
                    gray = (byte)((value - minValue) * 255.0f / (maxValue - minValue));
                } else 
                {
                    gray = (byte)(value >> 8);
                }
                
                int i = x * 4;
                dstRow[i + 0] = gray;
                dstRow[i + 1] = gray;
                dstRow[i + 2] = gray;
                dstRow[i + 3] = 255;
            }
        }
    }
    return dstBitmap;
}

SKBitmap Gray8ToGray8(int Width, int Height, int Stride, IntPtr bitmapData) {
    var dstInfo = new SKImageInfo(Width, Height, SKColorType.Gray8, SKAlphaType.Opaque);
    var dstBitmap = new SKBitmap(dstInfo);

    unsafe
    {
        byte* srcBase = (byte*)bitmapData;
        byte* dstBase = (byte*)dstBitmap.GetPixels().ToPointer();

        int dstStride = dstBitmap.RowBytes;

        for (int y = 0; y < Height; y++)
        {
            byte* srcRow = srcBase + y * Stride;
            byte* dstRow = dstBase + y * dstStride;

            Buffer.MemoryCopy(srcRow, dstRow, dstStride, Width);
        }
    }

    return dstBitmap;
}

SKBitmap Bgr24ToBgra8888(int Width, int Height, int Stride, IntPtr bitmapData, bool normalize = true) {
    var dstInfo = new SKImageInfo(Width, Height, SKColorType.Bgra8888, SKAlphaType.Opaque);
    var dstBitmap = new SKBitmap(dstInfo);

    unsafe
    {
        byte* srcBase = (byte*)bitmapData;
        byte* dstBase = (byte*)dstBitmap.GetPixels().ToPointer();

        int dstStride = dstBitmap.RowBytes;

        var bmin = byte.MaxValue;
        var bmax = byte.MinValue;
        var rmin = byte.MaxValue;
        var rmax = byte.MinValue;
        var gmin = byte.MaxValue;
        var gmax = byte.MinValue;

        if (normalize)
        {
            for (int y = 0; y < Height; y++)
            {
                byte* srcRow = srcBase + y * Stride;
                for (int x = 0; x < Width; x++) 
                {
                    int srcIndex = x * 3;
                    var b = srcRow[srcIndex + 0];
                    var g = srcRow[srcIndex + 1];
                    var r = srcRow[srcIndex + 2];
                    if (b < bmin) bmin = b;
                    if (b > bmax) bmax = b;
                    if (r < rmin) rmin = r;
                    if (r > rmax) rmax = r;
                    if (g < gmin) gmin = g;
                    if (g > gmax) gmax = g;
                }
            }
        }

        for (int y = 0; y < Height; y++)
        {
            byte* srcRow = srcBase + y * Stride;
            byte* dstRow = dstBase + y * dstStride;

            for (int x = 0; x < Width; x++) 
            {
                int srcIndex = x*3;
                int dstIndex = x*4;

                var b = srcRow[srcIndex + 0]; // B
                var g = srcRow[srcIndex + 1]; // G
                var r = srcRow[srcIndex + 2]; // R
                if (normalize) 
                {
                    if (bmin == bmax)
                    {
                        b = bmax;
                    } 
                    else 
                    {
                        b = (byte)((b-bmin)* 255.0f /(bmax-bmin));
                    }
                    
                    if (gmin == gmax)
                    {
                        g = gmax;
                    }
                    else 
                    {
                        g = (byte)((g-gmin)* 255.0f /(gmax-gmin));
                    }
                    if (rmin == rmax)
                    {
                        r = rmax;
                    }
                    else 
                    {
                        r = (byte)((r-rmin)* 255.0f /(rmax-rmin));
                    }
                }
                
                dstRow[dstIndex + 0] = b;
                dstRow[dstIndex + 1] = g;
                dstRow[dstIndex + 2] = r;
                dstRow[dstIndex + 3] = 255;
            }
        }
    }

    return dstBitmap;
}

SKBitmap ConvertIBitmapToSKBitmap(Zeiss.Micro.LibCzi.Net.Interface.IBitmap inBit, bool normalizeGray16 = true) {
    var lockInfo = inBit.Lock();
    SKBitmap bmp = null;
    switch (inBit.BitmapInfo.PixelType) {
        case PixelType.Gray16:
            bmp = Gray16ToBgra8888(inBit.BitmapInfo.Width, inBit.BitmapInfo.Height, lockInfo.Stride, lockInfo.BitmapData, normalizeGray16);
            break;
        case PixelType.Gray8:
            bmp = Gray8ToGray8(inBit.BitmapInfo.Width, inBit.BitmapInfo.Height, lockInfo.Stride, lockInfo.BitmapData);
            break;
        case PixelType.Bgr24:
            bmp = Bgr24ToBgra8888(inBit.BitmapInfo.Width, inBit.BitmapInfo.Height, lockInfo.Stride, lockInfo.BitmapData);
            break;
        default:
            break;
    }

    inBit.Unlock();
    return bmp;
}


### Pixel Conversion IBitmap -> ImageSharp

In [ ]:
using SixLabors.ImageSharp.PixelFormats;
using SixLabors.ImageSharp.Formats.Gif;
using SixLabors.ImageSharp.Processing;
using System.IO;

Image<Rgba32> Gray16ToISRgba32(int Width, int Height, int Stride, IntPtr bitmapData, bool normalize = true)
{
    var image = new Image<Rgba32>(Width, Height);

    unsafe
    {
        byte* srcBase = (byte*)bitmapData;

        ushort minValue = ushort.MaxValue;
        ushort maxValue = ushort.MinValue;

        if (normalize)
        {
            for (int y = 0; y < Height; y++) {
                ushort* srcRow = (ushort*)(srcBase + y * Stride);
                for (int x = 0; x < Width; x++) {
                    ushort value = srcRow[x];
                    if (value < minValue) minValue = value;
                    if (value > maxValue) maxValue = value;
                }
            }
        }
        
        image.ProcessPixelRows(accessor =>
        {
            for (int y = 0; y < accessor.Height; y++)
            {
                Span<Rgba32> row = accessor.GetRowSpan(y);
                ushort* srcRow = (ushort*)(srcBase + y * Stride);
                for (int x = 0; x < Width; x++) 
                {
                    ushort value = srcRow[x];
                    byte gray = 0;
                    if (normalize)
                    {
                        gray = (byte)((value - minValue) * 255.0f / (maxValue - minValue));
                    } else 
                    {
                        gray = (byte)(value >> 8);
                    }
                    ref Rgba32 pixel = ref row[x];
                    pixel.A = 255;
                    pixel.R = gray;
                    pixel.G = gray;
                    pixel.B = gray;
                }
            }                                          
        });            
    }
     
    return image;
}

Image<Rgba32> Gray8ToISRgba32(int Width, int Height, int Stride, IntPtr bitmapData)
{
    var image = new Image<Rgba32>(Width, Height);

    unsafe
    {
        byte* srcBase = (byte*)bitmapData;
        
        image.ProcessPixelRows(accessor =>
        {
            for (int y = 0; y < accessor.Height; y++)
            {
                byte* srcRow = (byte*)(srcBase + y * Stride);
                Span<Rgba32> row = accessor.GetRowSpan(y);
                for (int x = 0; x < Width; x++) 
                {
                    byte gray = srcRow[x];
                    ref Rgba32 pixel = ref row[x];
                    pixel.A = 255;
                    pixel.R = gray;
                    pixel.G = gray;
                    pixel.B = gray;
                }
            }                                          
        });            
    }
     
    return image;
}

Image<Rgba32> Bgr24ToISRgba32(int Width, int Height, int Stride, IntPtr bitmapData, bool normalize = true)
{
    var image = new Image<Rgba32>(Width, Height);

    unsafe
    {
        byte* srcBase = (byte*)bitmapData;

        var bmin = byte.MaxValue;
        var bmax = byte.MinValue;
        var rmin = byte.MaxValue;
        var rmax = byte.MinValue;
        var gmin = byte.MaxValue;
        var gmax = byte.MinValue;
                
        if (normalize)
        {
            for (int y = 0; y < Height; y++)
            {
                byte* srcRow = srcBase + y * Stride;
                for (int x = 0; x < Width; x++) 
                {
                    int srcIndex = x * 3;
                    var b = srcRow[srcIndex + 0];
                    var g = srcRow[srcIndex + 1];
                    var r = srcRow[srcIndex + 2];
                    if (b < bmin) bmin = b;
                    if (b > bmax) bmax = b;
                    if (r < rmin) rmin = r;
                    if (r > rmax) rmax = r;
                    if (g < gmin) gmin = g;
                    if (g > gmax) gmax = g;
                }
            }
        }
        
        image.ProcessPixelRows(accessor =>
        {
            for (int y = 0; y < accessor.Height; y++)
            {
                byte* srcRow = srcBase + y * Stride;
                Span<Rgba32> row = accessor.GetRowSpan(y);
                
                for (int x = 0; x < Width; x++) 
                {
                    int srcIndex = x * 3;
                    byte gray = srcRow[x];
                    ref Rgba32 pixel = ref row[x];

                    var b = srcRow[srcIndex + 0]; // B
                    var g = srcRow[srcIndex + 1]; // G
                    var r = srcRow[srcIndex + 2]; // R
                    if (normalize) 
                    {
                        if (bmin == bmax)
                        {
                            b = bmax;
                        } 
                        else 
                        {
                            b = (byte)((b-bmin)* 255.0f /(bmax-bmin));
                        }
                    
                        if (gmin == gmax)
                        {
                            g = gmax;
                        }
                        else 
                        {
                            g = (byte)((g-gmin)* 255.0f /(gmax-gmin));
                        }
                        if (rmin == rmax)
                        {
                            r = rmax;
                        }
                        else 
                        {
                            r = (byte)((r-rmin)* 255.0f /(rmax-rmin));
                        }
                    }
                
                    pixel.B = b;
                    pixel.G = g;
                    pixel.R = r;
                    pixel.A = 255;
                }
            }                                          
        });            
    }
     
    return image;
}
Image<Rgba32> ConvertIBitmapToISBitmap(Zeiss.Micro.LibCzi.Net.Interface.IBitmap inBit, bool normalizeGray16 = true) {
    var lockInfo = inBit.Lock();
    Image<Rgba32> bmp = null;
    switch (inBit.BitmapInfo.PixelType) {
        case PixelType.Gray16:
            bmp = Gray16ToISRgba32(inBit.BitmapInfo.Width, inBit.BitmapInfo.Height, lockInfo.Stride, lockInfo.BitmapData, normalizeGray16);
            break;
        case PixelType.Gray8:
            bmp = Gray8ToISRgba32(inBit.BitmapInfo.Width, inBit.BitmapInfo.Height, lockInfo.Stride, lockInfo.BitmapData);
            break;
        case PixelType.Bgr24:
            bmp = Bgr24ToISRgba32(inBit.BitmapInfo.Width, inBit.BitmapInfo.Height, lockInfo.Stride, lockInfo.BitmapData);
            break;
        default:
            break;
    }

    inBit.Unlock();
    return bmp;
}

### IBitmap Helpers

In [ ]:
string BitmapToHtmlImage(SKBitmap bmp) {
    using var image = SKImage.FromBitmap(bmp);
    using var encoded = image.Encode(SKEncodedImageFormat.Png, 100);
    var base64 = Convert.ToBase64String(encoded.ToArray());
    return $"<img src='data:image/png;base64,{base64}' />";
}

void DisplayIBitmap(Zeiss.Micro.LibCzi.Net.Interface.IBitmap inBitMap, bool normalize = true) {
    display(HTML(BitmapToHtmlImage(ConvertIBitmapToSKBitmap(inBitMap, normalize))));
}

### Pixel Conversion SKBitmap -> ImageSharp

In [ ]:
using SixLabors.ImageSharp.PixelFormats;
using SixLabors.ImageSharp.Formats.Gif;
using SixLabors.ImageSharp.Processing;
using System.IO;

byte[] CreateGifFromSKBitmaps(SKBitmap[] frames, int frameDelayMS = 100)
{
    using var gif = new Image<Rgba32>(frames[0].Width, frames[0].Height);

    for (int i = 0; i < frames.Length; i++) {
        var frame = frames[i];
        var image = new Image<Rgba32>(frame.Width, frame.Height);

        unsafe
        {
            byte* srcPtr = (byte*)frame.GetPixels().ToPointer();
            image.ProcessPixelRows(accessor =>
                {
                    for (int y = 0; y < accessor.Height; y++)
                    {
                        Span<Rgba32> row = accessor.GetRowSpan(y);
                        for (int x = 0; x < frame.Width; x++) {
                            ref Rgba32 pixel = ref row[x];
                            int offset = y * frame.RowBytes + x * 4;
                            pixel.A = srcPtr[offset + 3];
                            pixel.R = srcPtr[offset + 2];
                            pixel.G = srcPtr[offset + 1];
                            pixel.B = srcPtr[offset + 0];
                        }
                    }                                          
            });            
        }
        image.Frames.RootFrame.Metadata.GetGifMetadata().FrameDelay = frameDelayMS / 10;

        if (i == 0)
        {
            gif.Mutate(x => x.DrawImage(image, 1));
        }
        else
        {
            image.Frames.RootFrame.Metadata.GetGifMetadata().FrameDelay = frameDelayMS / 10;
            gif.Frames.AddFrame(image.Frames.RootFrame);
        }
    }

    using var ms = new MemoryStream();
    gif.Metadata.GetGifMetadata().RepeatCount = 0;
    gif.SaveAsGif(ms);
    return ms.ToArray();
}
byte[] CreateGifFromISBitmaps(Image<Rgba32>[] frames, int frameDelayMS = 100)
{
    using var gif = new Image<Rgba32>(frames[0].Width, frames[0].Height);

    for (int i = 0; i < frames.Length; i++) {
        var frame = frames[i];
        var image = frame;
        image.Frames.RootFrame.Metadata.GetGifMetadata().FrameDelay = frameDelayMS / 10;

        if (i == 0)
        {
            gif.Mutate(x => x.DrawImage(image, 1));
        }
        else
        {
            gif.Frames.AddFrame(image.Frames.RootFrame);
        }
    }

    using var ms = new MemoryStream();
    gif.Metadata.GetGifMetadata().RepeatCount = 0;
    gif.SaveAsGif(ms);
    return ms.ToArray();
}

void DisplayGif(Byte[] gifData)
{
    var base64Data = Convert.ToBase64String(gifData);
    display(HTML($"<img src='data:image/gif;base64,{base64Data}' />"));
}

In [ ]:
using Microsoft.DotNet.Interactive;

void inPlaceImgDisp(SKBitmap[] frames, int frameDelay = 20, bool loopForever = false)
{
    var imgOutput = display(frames[0]);
    do {
        for (int i = 0; i < frames.Length; i++) {
            System.Threading.Thread.Sleep(frameDelay);
            imgOutput.Update(frames[i]);
        }
    } while (loopForever);

}